In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def plot_from_csv(csv_file, folder, x_column, y_columns, legend_column, split_by):
    """
    Plots each Y column vs X column.
    One chart is made per `split_by` group (e.g. forwardingStrategy).
    Legend is based on `legend_column` (e.g. dropPolicy).
    Charts are saved inside: data/{folder}/charts/{split_by_value}/...
    """
    df = pd.read_csv(csv_file)

    # Drop rows with missing values in important columns
    required_columns = [x_column, legend_column, split_by] + y_columns
    df = df.dropna(subset=required_columns)

    # Helper: Convert "10M" → 10000000 for sorting X-axis
    def convert_x(x):
        try:
            return float(x)
        except ValueError:
            match = re.search(r'(\d+(?:\.\d+)?)([KkMmGg]?)', str(x))
            if not match:
                return float('inf')
            num, suffix = match.groups()
            num = float(num)
            scale = {'K': 1e3, 'M': 1e6, 'G': 1e9}
            return num * scale.get(suffix.upper(), 1)

    # Add sortable column for X
    df['_x_numeric'] = df[x_column].apply(convert_x)

    # Create base output directory
    base_dir = os.path.join("data", folder, "charts")
    os.makedirs(base_dir, exist_ok=True)

    # Split by forwardingStrategy (or your split_by field)
    for strategy, subset in df.groupby(split_by):
        subset = subset.sort_values('_x_numeric')

        # Create subfolder for this strategy
        safe_strategy = re.sub(r'[^\w\-]+', '_', str(strategy))  # sanitize
        strategy_dir = os.path.join(base_dir, safe_strategy)
        os.makedirs(strategy_dir, exist_ok=True)

        for y_col in y_columns:
            if y_col not in subset.columns:
                print(f"⚠️ Skipping missing column: {y_col}")
                continue

            plt.figure(figsize=(10, 6))
            sns.lineplot(
                data=subset,
                x=x_column,
                y=y_col,
                hue=legend_column,
                marker='o'
            )

            plt.title(f"{y_col} vs {x_column} — {split_by}: {strategy}")
            plt.xlabel(x_column)
            plt.ylabel(y_col)
            plt.legend(title=legend_column)
            plt.grid(True)
            plt.tight_layout()

            # Save chart
            filename = os.path.join(strategy_dir, f"{y_col}_vs_{x_column}_{safe_strategy}.png")
            plt.savefig(filename)
            plt.close()
            print(f"✅ Chart saved: {filename}")

In [ ]:
def plot_from_csv(csv_file, folder, x_column, y_columns, legend_column):
    """
    Plots one or more y_columns against a selected x_column from the CSV,
    and uses legend_column for line groups.
    """
    df = pd.read_csv(csv_file)

    # Drop rows with missing values in any of the required columns
    df = df.dropna(subset=[x_column, legend_column] + y_columns)

    # Try converting X values to sortable numeric form (e.g., "10M" -> 10000000)
    def convert_x(x):
        try:
            return float(x)
        except ValueError:
            match = re.search(r'(\d+(?:\.\d+)?)([KkMmGg]?)', str(x))
            if not match:
                return float('inf')  # If conversion fails
            num, suffix = match.groups()
            num = float(num)
            scale = {'K': 1e3, 'M': 1e6, 'G': 1e9}
            return num * scale.get(suffix.upper(), 1)

    # Add numeric column for sorting
    df['_x_numeric'] = df[x_column].apply(convert_x)
    df = df.sort_values('_x_numeric')

    # Ensure charts folder exists
    chart_dir = os.path.join("data", folder, "charts")
    os.makedirs(chart_dir, exist_ok=True)

    for y_col in y_columns:
        if y_col not in df.columns:
            print(f"⚠️ Warning: '{y_col}' not found in CSV. Skipping.")
            continue

        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=df,
            x=x_column,
            y=y_col,
            hue=legend_column,
            marker='o'
        )

        plt.title(f"{y_col} vs {x_column}")
        plt.xlabel(x_column)
        plt.ylabel(y_col)
        plt.legend(title=legend_column)
        plt.grid(True)
        plt.tight_layout()

        # Save the plot
        filename = os.path.join(chart_dir, f"{y_col}_vs_{x_column}.png")
        plt.savefig(filename)
        plt.close()
        print(f"✅ Chart saved: {filename}")

In [ ]:
def plot_from_csv(csv_file, folder, x_column, y_columns, legend_column):
    # Load data
    df = pd.read_csv(csv_file)
    
    # Drop rows where x_column or legend_column is missing
    df = df.dropna(subset=[x_column, legend_column])
    
    # Ensure proper sorting and type conversion
    df[x_column] = df[x_column].astype(str)
    df[legend_column] = df[legend_column].astype(str)

    for y_col in y_columns:
        if y_col not in df.columns:
            print(f"⚠️ Warning: '{y_col}' not found in CSV. Skipping.")
            continue
        
        # Drop rows with missing or NaN Y values
        chart_df = df.dropna(subset=[y_col])

        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=chart_df,
            x=x_column,
            y=y_col,
            hue=legend_column,
            marker='o'
        )

        plt.title(f"{y_col} vs {x_column}")
        plt.xlabel(x_column)
        plt.ylabel(y_col)
        plt.legend(title=legend_column)
        plt.grid(True)
        plt.tight_layout()

        # Create output folder
        chart_dir = os.path.join("data", folder, "charts")
        os.makedirs(chart_dir, exist_ok=True)

        filename = os.path.join(chart_dir, f"{y_col}_vs_{x_column}.png")
        plt.savefig(filename)

        plt.close()
        print(f"✅ Chart saved: {filename}")

In [4]:
plot_from_csv(
    csv_file='data/gridProphetMulti/csv/summary_report.csv',
    folder='gridProphetMulti',
    x_column='bufferSize',
    y_columns=['delivery_prob', 'latency_avg', 'overhead_ratio', 'hopcount_avg'],
    legend_column='dropPolicy',
    split_by="forwardingStrategy"
)

✅ Chart saved: data\gridProphetMulti\charts\COIN\delivery_prob_vs_bufferSize_COIN.png
✅ Chart saved: data\gridProphetMulti\charts\COIN\latency_avg_vs_bufferSize_COIN.png
✅ Chart saved: data\gridProphetMulti\charts\COIN\overhead_ratio_vs_bufferSize_COIN.png
✅ Chart saved: data\gridProphetMulti\charts\COIN\hopcount_avg_vs_bufferSize_COIN.png
✅ Chart saved: data\gridProphetMulti\charts\GRTR\delivery_prob_vs_bufferSize_GRTR.png
✅ Chart saved: data\gridProphetMulti\charts\GRTR\latency_avg_vs_bufferSize_GRTR.png
✅ Chart saved: data\gridProphetMulti\charts\GRTR\overhead_ratio_vs_bufferSize_GRTR.png
✅ Chart saved: data\gridProphetMulti\charts\GRTR\hopcount_avg_vs_bufferSize_GRTR.png
✅ Chart saved: data\gridProphetMulti\charts\GRTRMax\delivery_prob_vs_bufferSize_GRTRMax.png
✅ Chart saved: data\gridProphetMulti\charts\GRTRMax\latency_avg_vs_bufferSize_GRTRMax.png
✅ Chart saved: data\gridProphetMulti\charts\GRTRMax\overhead_ratio_vs_bufferSize_GRTRMax.png
✅ Chart saved: data\gridProphetMulti\char